# Interacting with the knowledge graph

Knowledge graph plays a key role in motleycrew. It is used to store the state that is used to dispatch workers, plus any other state you wish to store and query as part of your application.

We are currently using [kuzu](https://github.com/kuzudb) as the knowledge graph backend, because it's embeddable, supports openCypher and is available under the MIT license, and also has [LlamaIndex integration](https://docs.llamaindex.ai/en/stable/api_reference/storage/graph_stores/kuzu/); please let us know if you would like to use another backend.

To make interaction with kuzu from Python more natural, we have written a thin OGM (Object-graph management) layer on top of kuzu; it also allows you to do an arbitrary Cypher query to kuzu if its abstractions don't fit your purpose.